# Rigid Bodies

In [1]:
import os 

import foyer
import hoomd
from hoomd.data import make_snapshot
import mbuild as mb
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import numpy as np
import matplotlib.pyplot as plt

from planckton.utils.rigid import connect_rings, moit, init_rigid
from planckton.utils.utils import set_coeffs
from planckton.init import Compound, Pack

/Users/jenny/miniconda3/envs/planckton-dev/lib/python3.7/site-packages/mdtraj/core/trajectory.py:32: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/Users/jenny/miniconda3/envs/planckton-dev/lib/python3.7/site-packages/foyer/forcefield.py:449: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/Users/jenny/miniconda3/envs/planckton-dev/lib/python3.7/site-packages/foyer/forcefield.py:461: UserWarning: No force field name found in force field XML file.
  'No force field name found in force field XML file.'
/Users/jenny/miniconda3/envs/planckton-dev/lib/python3.7/site-packages/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)


# trying to modify planckton's init and sim functions

In [3]:
input_str = "c1ccccc1"
#input_str = "planckton/compounds/CZTPTZITIC_typed.mol2"

if os.path.exists(input_str):
    comp = mb.load(input_str)
else:
    comp = mb.load(input_str, smiles=True)

# Calculate mass of compound                                            
comp.mass = np.sum([atom.mass for atom in comp.to_parmed().atoms])      
                                                                        
# This helps to_parmed use residues to apply ff more quickly            
comp.name = os.path.basename(input_str).split(".")[0] 
print(comp.mass, comp.name)

78.11160000000002 c1ccccc1


/Users/jenny/miniconda3/envs/planckton-dev/lib/python3.7/site-packages/mbuild/conversion.py:690: UserWarning: No unitcell detected for pybel.Molecule c1ccccc1	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


In [4]:
density = 0.01
packer = Pack(comp, 10, density)
L=packer.L
print(L)
typed_system, rigid_system = packer.pack()

5.061993029949088


In [5]:
sim = hoomd.context.initialize("")
both, snap, sim, ref_values = init_rigid(rigid_system, typed_system, sim)

HOOMD-blue 2.9.3 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 10/17/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 130 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10
notice(2): Particles with 3 exclusions             : 60
notice(2): Particles with 7 exclusions             : 60
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
Processing LJ and QQ
notice(2): Group "charged" created containing 0 particles
No charged groups found, ignoring electrostatics
Processing 1-4 interactions, adj

In [6]:
del(both)

In [7]:
target_length = 10
e_factor = 0.5
#for force in sim.forces:
#    try:
#        print(force.pair_coeff.values,"\n")
#    except AttributeError:
#        pass
with sim:                                                               
    if target_length is not None:                                  
        target_length /= ref_values.distance                       
                                                                        
    if e_factor != 1:                                                             
        hoomd.util.quiet_status()                                       
        # catch all instances of LJ pair                                
        ljtypes = [                                                     
                i for i in sim.forces                                   
                if isinstance(i, hoomd.md.pair.lj)                      
                or isinstance(i, hoomd.md.special_pair.lj)              
                ] 
        for lj in ljtypes:                                              
            pair_list = lj.get_metadata()['pair_coeff'].get_metadata()  
            for pair_dict in pair_list:                                 
                # Scale the epsilon values by e_factor                  
                try:                                                    
                    a, b, new_dict = set_coeffs(                        
                            pair_dict,                                  
                            e_factor                               
                            )                                           
                    lj.pair_coeff.set(a, b, **new_dict)                 
                except ValueError:                                      
                    # if the pair has not been defined,                 
                    # it will not have a dictionary object              
                    # instead it will be a string (e.g. "ca-ca")        
                    # and will fail when trying to make the new_dict    
                    pass       

    #for force in sim.forces:
    #    try:
    #        print(force.pair_coeff.values,"\n")
    #    except AttributeError:
    #        pass
    
    integrator_mode = hoomd.md.integrate.mode_standard(dt=0.001)      
    all_particles = hoomd.group.all()                          
    try:
        integrator = hoomd.md.integrate.nvt(                                
            group=both, tau=1, kT=1    
        )          
    except NameError:
        # both does not exist
        integrator = hoomd.md.integrate.nvt(                                
            group=all_particles, tau=1, kT=1    
        )       

**ERROR**: Particle 10 belongs to a rigid body, but is not its center particle. 
This integration method does not operate on constituent particles.



RuntimeError: Error initializing integration method

In [7]:
with sim:
    hoomd.dump.gsd(                                                     
        filename="trajectory.gsd",                                      
        period=1e3,                                          
        group=all_particles,                                            
        overwrite=False,                                                
        phase=0,                                                        
    )  
    integrator.randomize_velocities(seed=42)
    hoomd.run(1e4)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10
notice(2): Particles with 7 exclusions             : 60
notice(2): Particles with 10 exclusions             : 60
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:01 | Step 10000 / 10000 | TPS 7070.07 | ETA 00:00:00
Average TPS: 7065.34
---------
-- Neighborlist stats:
100 normal updates / 100 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 5 / n_neigh_avg: 1.61538
shortest rebuild period: 50
-- Cell list stats:
Dimension: 46, 46, 46
n_min    : 0 / n_max: 13 / n_avg: 0.00133558
** run complete **


# Working example with Rigid Bodies

Let's start with a simple case--a benzene molcule.

In [6]:
bz_str = "c1ccccc1"
bz = mb.load(bz_str, smiles=True, ignore_box_warn=True)
name = "bz"
bz.name = name
#bz.visualize().show()

In [7]:
#npt_str = "c1ccc2ccccc2c1"
#npt = mb.load(npt_str, smiles=True, ignore_box_warn=True)
#npt.visualize().show()

In [8]:
#py_str = "c1cc2cccc3ccc4cccc1c4c32"
#py = mb.load(py_str, smiles=True, ignore_box_warn=True)
#py.visualize().show()

Let's try fixing the orientation of the benzene molecules when we fill the box--I think that might make it work better with rigid bodies.

In [9]:
box = mb.Box([5,5,5])
system = mb.fill_box(bz, n_compounds=10, box=box, fix_orientation=True)
#system.visualize().show()

In [10]:
gaff = foyer.forcefields.load_GAFF()
pmd_system = system.to_parmed(residues=[name])
typed_system = gaff.apply(system)
#print(set([atom.type for atom in typed_system.atoms]))


I'm using mbuild from [PR #808](https://github.com/mosdef-hub/mbuild/pull/808) which allows `create_hoomd_simulation` to read in a snapshot:

```
create_hoomd_simulation(structure, ref_distance=1.0, ref_mass=1.0, ref_energy=1.0, r_cut=1.2, auto_scale=False, snapshot_kwargs={}, pppm_kwargs={'Nx': 8, 'Ny': 8, 'Nz': 8, 'order': 4}, init_snap=None)
```

First, make a snapshot with 10 rigid particles--one for each benzene ring:

In [11]:
sim = hoomd.context.SimulationContext()

with sim:
    hoomd.context.initialize("")
    init_snap = make_snapshot(N=10, particle_types=["_R"], box=hoomd.data.boxdim(L=10))

In [12]:
with sim:
    hoomd_objects, ref_values = create_hoomd_simulation(
        typed_system, auto_scale=True, init_snap=init_snap
    )
    snap = hoomd_objects[0]

notice(2): Group "all" created containing 130 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10
notice(2): Particles with 3 exclusions             : 60
notice(2): Particles with 7 exclusions             : 60
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
Processing LJ and QQ
notice(2): Group "charged" created containing 0 particles
No charged groups found, ignoring electrostatics
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions
HOOMD SimulationContext updated from ParmEd Structure


Want to do something with [ring detection](https://openbabel.org/wiki/Ring_detection) where rings are automatically converted to rigid bodies.

- convert to pybel mol
- use smarts matching to find rings
- make rigid

[SSSR documentation](http://openbabel.org/dev-api/classOpenBabel_1_1OBRing.shtml#_details)

In [13]:
system_mol = system.to_pybel()
rings = sorted(connect_rings(system_mol), key=lambda x: x[0])

#print(*rings, sep="\n")
print(len(rings))

10


Now let's move the rigid body centers to the center of the ring and set the body IDs

In [10]:
for i,ring in enumerate(rings):
    inds = ring + len(rings)
    snap.particles.position[i] = np.mean(snap.particles.position[inds], axis=0)
    snap.particles.body[i] = i
    snap.particles.body[inds] = i * np.ones(len(ring))

I am using [this example](http://farside.ph.utexas.edu/teaching/336k/Newtonhtml/node64.html) for how to calculate the moment of inertia tensor (also Matty/Mike/someone's code from cmeutils)

From [hoomd docs](https://hoomd-blue.readthedocs.io/en/v2.9.3/module-md-constrain.html#hoomd.md.constrain.rigid)
>The mass and moment of inertia of the central particle set the full mass and moment of inertia of the rigid body (constituent particle mass is ignored).

>The central particle is at the center of mass of the rigid body and the orientation quaternion defines the rotation from the body space into the simulation box. In body space, the center of mass of the body is at 0,0,0 and the moment of inertia is diagonal. 

In [11]:
for i,ring in enumerate(rings):
    inds = ring + len(rings)
    snap.particles.moment_inertia[i] = moit(
        snap.particles.position[inds], snap.particles.mass[inds], center=snap.particles.position[i]
    )
    snap.particles.mass[i] = np.sum(snap.particles.mass[inds])

In [12]:
# need to reinitialize
sim.system_definition.initializeFromSnapshot(snap)

In [13]:
nl = sim.neighbor_lists[0]
ex_list = nl.exclusions 
ex_list.append('body')
sim.neighbor_lists[0].reset_exclusions(exclusions=ex_list)

print(sim.neighbor_lists[0].exclusions)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10
notice(2): Particles with 7 exclusions             : 60
notice(2): Particles with 10 exclusions             : 60
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
['1-2', '1-3', '1-4', 'body']


In [14]:
with sim:
    rigid = hoomd.md.constrain.rigid()
    
    r_pos = snap.particles.position[0]
    const_pos = snap.particles.position[rings[0]+len(rings)]
    const_pos -= r_pos
    #print(r_pos,const_pos)
    
    const_types = [snap.particles.types[i] for i in snap.particles.typeid[rings[0]+len(rings)]]
    #print(const_types)
    
    rigid.set_param("_R", types=const_types, positions=[tuple(i) for i in const_pos])
    rigid.validate_bodies()
    
    lj = sim.forces[0]
    lj.pair_coeff.set("_R", snap.particles.types, epsilon=0, sigma=0)
    
    centers = hoomd.group.rigid_center()
    nonrigid = hoomd.group.nonrigid()
    _all = hoomd.group.all()
    
    hoomd.md.integrate.mode_standard(dt=0.0001);
    hoomd.md.integrate.langevin(group=centers, kT=1.0, seed=42);
    hoomd.md.integrate.langevin(group=nonrigid, kT=1.0, seed=42);
    hoomd.dump.gsd(filename="start.gsd", overwrite=True, period=None, group=_all, time_step=0)
    hoomd.dump.gsd("trajectory.gsd", period=1e3, group=_all, overwrite=True)
    hoomd.run(5e4)

notice(2): Group "rigid_center" created containing 10 particles
notice(2): Group "nonrigid" created containing 60 particles
notice(2): integrate.langevin/bd is using specified gamma values
notice(2): integrate.langevin/bd is using specified gamma values
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10
notice(2): Particles with 7 exclusions             : 60
notice(2): Particles with 10 exclusions             : 60
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **


*Warning*: Integrator #1: Anisotropic integration requested, but no rotational degrees of freedom found for its group


Time 00:00:20 | Step 50000 / 50000 | TPS 9010.01 | ETA 00:00:00
Average TPS: 9008.75
---------
-- Neighborlist stats:
0 normal updates / 500 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 6 / n_neigh_avg: 1.61538
shortest rebuild period: 100
-- Cell list stats:
Dimension: 9, 9, 9
n_min    : 0 / n_max: 12 / n_avg: 0.178326
** run complete **


In [15]:
with sim:
    hoomd.dump.gsd(filename="after.gsd", overwrite=True, period=None, group=hoomd.group.all())